In [1]:
import numpy as np
import pandas as pd

In [6]:
train_df = pd.read_csv('hw1data/propublicaTrain.csv')
test_df = pd.read_csv('hw1data/propublicaTest.csv')

In [10]:
train_df.head()

,two_year_recid,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree_F,c_charge_degree_M
0,0,1,64,0,0,0,0,13,0,1
1,0,1,28,0,0,0,0,1,1,0
2,0,1,32,0,0,0,0,1,1,0
3,1,1,20,0,0,1,1,2,1,0
4,0,1,43,1,0,0,0,8,1,0


In [11]:
train_df.describe()

,two_year_recid,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree_F,c_charge_degree_M
count,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000
mean,0.455964,0.813775,34.409167,0.344612,0.055916,0.089513,0.110631,3.278138,0.645308,0.354692
std,0.498117,0.389335,11.570853,0.475299,0.379495,0.486893,0.464278,4.770756,0.478477,0.478477
min,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,31.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,1.000000,1.000000,42.000000,1.000000,0.000000,0.000000,0.000000,4.000000,1.000000,1.000000
max,1.000000,1.000000,83.000000,1.000000,8.000000,13.000000,7.000000,38.000000,1.000000,1.000000


In [12]:
test_df.describe()

,two_year_recid,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree_F,c_charge_degree_M
count,2000.00000,2000.00000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.45450,0.80150,34.787500,0.332000,0.06600,0.095000,0.111000,3.183500,0.638500,0.361500
std,0.49805,0.39897,12.042253,0.471049,0.60235,0.520683,0.484558,4.692423,0.480555,0.480555
min,0.00000,0.00000,19.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,1.00000,25.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,1.00000,31.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,1.00000,1.00000,42.000000,1.000000,0.00000,0.000000,0.000000,4.000000,1.000000,1.000000
max,1.00000,1.00000,96.000000,1.000000,20.00000,12.000000,9.000000,36.000000,1.000000,1.000000


NOTE:
- two_year_recid (Y), sex, race, c_charge_degree_F are categorical
- c_charge_degree_M is ignored because c_charge_degree_F includes all information about the column

# Training

### Mutlivariate Guassian Distibution fitting

In [ ]:
def fit_guassian()

### Non-recidivism estimation

### Recidivism estimation

### Categorical Feature as joint conditional probabilities

# Prediction: Take argmax of the two estimations